In [3]:
import tensorflow as tf

In [5]:
%ls data

binary_tweets.csv      parsed_start_462.json  tweets_parsed_2.json
first_scrape.json      README.md              tweets_parsed.json
multiclass_tweets.csv  tweets.json
parsed_start_0.json    tweets_normalized.txt


In [21]:
import pandas as pd
df = pd.read_json('data/parsed_start_0.json')

In [57]:
len(df.dropna(subset=['output']))

116882

In [63]:
print(len(df[df['output'].isnull()]) == len(df[df['output'].isnull() & df['input'].isnull()]))
df = df.dropna()

True


In [89]:
def dict_get(key):
  if isinstance(key, str):
    return lambda i: i[key]
  return lambda i: [v for k, v in i.items() if k in key]

as_list = lambda v: v if isinstance(v, list) else [v]
def dict_filter(key):
  key = as_list(key)
  return lambda i: dict(filter(lambda kv: kv[0] in key, i.items()))

df_text = dict_get('input')
df_parse = dict_get('output')

out_word = dict_get('word')
out_pos = dict_get('pos_tag')

In [90]:
dict_filter(['category', 'dep'])(df_parse(df.iloc[0])[0])

{'category': '', 'dep': 'advmod'}